In [161]:
from typing import Tuple

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch import Tensor
from torch.utils.data import Dataset, DataLoader,TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report
from sklearn.impute import SimpleImputer

import matplotlib.pyplot as plt

In [162]:
df = pd.read_csv(r"C:\Users\Admin\Downloads\student_admission_record_dirty.csv")

In [163]:
df

,Name,Age,Gender,Admission Test Score,High School Percentage,City,Admission Status
0,Shehroz,24.0,Female,50.0,68.90,Quetta,Rejected
1,Waqar,21.0,Female,99.0,60.73,Karachi,NaN
2,Bushra,17.0,Male,89.0,NaN,Islamabad,Accepted
3,Aliya,17.0,Male,55.0,85.29,Karachi,Rejected
4,Bilal,20.0,Male,65.0,61.13,Lahore,NaN
...,...,...,...,...,...,...,...
152,Ali,19.0,Female,85.0,78.09,Quetta,Accepted
153,Bilal,17.0,Female,81.0,84.40,Islamabad,Rejected
154,Fatima,21.0,Female,98.0,50.86,Multan,Accepted
155,Shoaib,-1.0,Male,91.0,80.12,Quetta,Accepted


In [127]:
df.isnull().sum()

Name                      10
Age                       10
Gender                    10
Admission Test Score      11
High School Percentage    11
City                      10
Admission Status          10
dtype: int64

In [128]:
df = df.drop(columns="Name")

In [129]:
df.loc[df["Age"]<0,"Age"] = np.nan

In [130]:
df = df.dropna(subset=["Admission Status"])


In [131]:
X = df.drop(columns="Admission Status")
y = df["Admission Status"]

In [132]:
label = LabelEncoder()
y_encoded = label.fit_transform(y)

In [133]:
num_cols = ["Age","Admission Test Score","High School Percentage"]
cat_cols = ["Gender","City"]

In [134]:
num_imputer = SimpleImputer(strategy="median")
col_imputer = SimpleImputer(strategy="most_frequent")

df[num_cols] = num_imputer.fit_transform(df[num_cols])
df[cat_cols] = col_imputer.fit_transform(df[cat_cols])

In [135]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num",StandardScaler(),num_cols),
        ("cat",OneHotEncoder(drop="first"),cat_cols)
    ]
)

In [136]:
X_train_val,X_test,y_train_val,y_test = train_test_split(X,y_encoded,test_size=0.2,stratify=y_encoded)

test_size = 0.2
size_ratio = test_size / (1-0.2)

X_train, X_val ,y_train,y_val = train_test_split(X_train_val,y_train_val,test_size=size_ratio,stratify=y_train_val)

In [137]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)
X_val = preprocessor.transform(X_val)

In [138]:
X_train = torch.tensor(X_train,dtype=torch.float32)
X_test = torch.tensor(X_test,dtype=torch.float32)
X_val = torch.tensor(X_val,dtype=torch.float32)

y_train = torch.tensor(y_train,dtype=torch.long)
y_test = torch.tensor(y_test,dtype=torch.long)
y_val = torch.tensor(y_val,dtype=torch.long)

In [139]:
train_ds = TensorDataset(X_train,y_train)
test_ds = TensorDataset(X_test,y_test)
val_ds = TensorDataset(X_val,y_val)

train_loader = DataLoader(train_ds,batch_size=32)
test_loader = DataLoader(test_ds,batch_size=32)
val_loader = DataLoader(val_ds,batch_size=32)

In [140]:
import torch.nn as nn
input_dim = X_train.shape[1]
model = nn.Sequential(
        nn.Linear(input_dim, 64),
        nn.ReLU(),
        nn.Linear(64, 32),
        nn.ReLU(),
        nn.Linear(32, 1)  
    )
       

In [141]:
criterion = nn.BCEWithLogitsLoss()
optimizer =  torch.optim.Adam(model.parameters(),lr=0.001)

In [142]:
df.isnull().sum()

Age                       0
Gender                    0
Admission Test Score      0
High School Percentage    0
City                      0
Admission Status          0
dtype: int64

In [143]:
epochs = 50

for epoch in range(epochs):
    model.train()
    train_loss = 0.0

    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs.squeeze(1), y_batch.float())
        loss.backward()     
        optimizer.step()
        train_loss += loss.item() 

    avg_loss = train_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")


Epoch 1/50, Loss: nan
Epoch 2/50, Loss: nan
Epoch 3/50, Loss: nan
Epoch 4/50, Loss: nan
Epoch 5/50, Loss: nan
Epoch 6/50, Loss: nan
Epoch 7/50, Loss: nan
Epoch 8/50, Loss: nan
Epoch 9/50, Loss: nan
Epoch 10/50, Loss: nan
Epoch 11/50, Loss: nan
Epoch 12/50, Loss: nan
Epoch 13/50, Loss: nan
Epoch 14/50, Loss: nan
Epoch 15/50, Loss: nan
Epoch 16/50, Loss: nan
Epoch 17/50, Loss: nan
Epoch 18/50, Loss: nan
Epoch 19/50, Loss: nan
Epoch 20/50, Loss: nan
Epoch 21/50, Loss: nan
Epoch 22/50, Loss: nan
Epoch 23/50, Loss: nan
Epoch 24/50, Loss: nan
Epoch 25/50, Loss: nan
Epoch 26/50, Loss: nan
Epoch 27/50, Loss: nan
Epoch 28/50, Loss: nan
Epoch 29/50, Loss: nan
Epoch 30/50, Loss: nan
Epoch 31/50, Loss: nan
Epoch 32/50, Loss: nan
Epoch 33/50, Loss: nan
Epoch 34/50, Loss: nan
Epoch 35/50, Loss: nan
Epoch 36/50, Loss: nan
Epoch 37/50, Loss: nan
Epoch 38/50, Loss: nan
Epoch 39/50, Loss: nan
Epoch 40/50, Loss: nan
Epoch 41/50, Loss: nan
Epoch 42/50, Loss: nan
Epoch 43/50, Loss: nan
Epoch 44/50, Loss: n

In [164]:
from sklearn.pipeline import Pipeline

df = df.drop(columns="Name")
df.loc[df["Age"] < 0, "Age"] = np.nan
df = df.dropna(subset=["Admission Status"])

X = df.drop(columns="Admission Status")
y = df["Admission Status"]

label = LabelEncoder()
y_encoded = label.fit_transform(y)

num_cols = ["Age","Admission Test Score","High School Percentage"]
cat_cols = ["Gender","City"]

preprocessor = ColumnTransformer(
    transformers=[
        ("num", Pipeline([
            ("imputer", SimpleImputer(strategy="median")),
            ("scaler", StandardScaler())
        ]), num_cols),
        
        ("cat", Pipeline([
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("onehot", OneHotEncoder(drop="first"))
        ]), cat_cols),
    ]
)

# ---------- Splits ----------
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42
)

size_ratio = 0.2 / 0.8

X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val,
    test_size=size_ratio,
    stratify=y_train_val,
    random_state=42
)

# ---------- Fit ONLY on train ----------
X_train = preprocessor.fit_transform(X_train)
X_val   = preprocessor.transform(X_val)
X_test  = preprocessor.transform(X_test)

# ---------- To tensors ----------
X_train = torch.tensor(X_train, dtype=torch.float32)
X_val   = torch.tensor(X_val, dtype=torch.float32)
X_test  = torch.tensor(X_test, dtype=torch.float32)

y_train = torch.tensor(y_train, dtype=torch.float32)
y_val   = torch.tensor(y_val, dtype=torch.float32)
y_test  = torch.tensor(y_test, dtype=torch.float32)

train_loader = DataLoader(TensorDataset(X_train,y_train), batch_size=32)
val_loader   = DataLoader(TensorDataset(X_val,y_val), batch_size=32)
test_loader  = DataLoader(TensorDataset(X_test,y_test), batch_size=32)

# ---------- Model ----------
input_dim = X_train.shape[1]
model = nn.Sequential(
    nn.Linear(input_dim,64),
    nn.ReLU(),
    nn.Linear(64,32),
    nn.ReLU(),
    nn.Linear(32,1)
)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

def get_accuracy(outputs, targets):
    probs = torch.sigmoid(outputs)
    preds = (probs >= 0.5).float()
    return (preds == targets).float().mean().item()


# ---------- Training ----------
epochs = 50

for epoch in range(epochs):
    # ---------- TRAIN ----------
    model.train()
    train_loss = 0
    train_acc = 0

    for xb, yb in train_loader:
        optimizer.zero_grad()

        outputs = model(xb).squeeze(1)

        loss = criterion(outputs, yb)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_acc += get_accuracy(outputs, yb)

    train_loss /= len(train_loader)
    train_acc /= len(train_loader)

    # ---------- VALIDATION ----------
    model.eval()
    val_loss = 0
    val_acc = 0

    with torch.no_grad():
        for xb, yb in val_loader:
            outputs = model(xb).squeeze(1)
            loss = criterion(outputs, yb)

            val_loss += loss.item()
            val_acc += get_accuracy(outputs, yb)

    val_loss /= len(val_loader)
    val_acc /= len(val_loader)

    print(
        f"Epoch {epoch+1}/{epochs} "
        f"| Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} "
        f"| Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}"
    )



Epoch 1/50 | Train Loss: 0.6945 | Train Acc: 0.5118 | Val Loss: 0.7006 | Val Acc: 0.4667
Epoch 2/50 | Train Loss: 0.6908 | Train Acc: 0.4909 | Val Loss: 0.7014 | Val Acc: 0.4667
Epoch 3/50 | Train Loss: 0.6886 | Train Acc: 0.5014 | Val Loss: 0.7020 | Val Acc: 0.4667
Epoch 4/50 | Train Loss: 0.6867 | Train Acc: 0.5118 | Val Loss: 0.7026 | Val Acc: 0.4333
Epoch 5/50 | Train Loss: 0.6849 | Train Acc: 0.5326 | Val Loss: 0.7032 | Val Acc: 0.4333
Epoch 6/50 | Train Loss: 0.6831 | Train Acc: 0.5575 | Val Loss: 0.7039 | Val Acc: 0.4333
Epoch 7/50 | Train Loss: 0.6814 | Train Acc: 0.5367 | Val Loss: 0.7047 | Val Acc: 0.4333
Epoch 8/50 | Train Loss: 0.6797 | Train Acc: 0.5471 | Val Loss: 0.7055 | Val Acc: 0.4333
Epoch 9/50 | Train Loss: 0.6779 | Train Acc: 0.5720 | Val Loss: 0.7065 | Val Acc: 0.4333
Epoch 10/50 | Train Loss: 0.6761 | Train Acc: 0.6259 | Val Loss: 0.7076 | Val Acc: 0.4667
Epoch 11/50 | Train Loss: 0.6742 | Train Acc: 0.6259 | Val Loss: 0.7087 | Val Acc: 0.4000
Epoch 12/50 | Train

In [165]:
model.eval()
test_loss = 0
test_acc = 0

with torch.no_grad():
    for xb, yb in test_loader:
        outputs = model(xb).squeeze(1)
        loss = criterion(outputs, yb)

        test_loss += loss.item()
        test_acc += get_accuracy(outputs, yb)

test_loss /= len(test_loader)
test_acc /= len(test_loader)

print(f"Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.4f}")


Test Loss: 0.8550 | Test Accuracy: 0.4667
